In [ ]:
import sys
import time
import pygame

# Initialize pygame
pygame.init()

# Define screen size
width, height = 600, 400
screen = pygame.display.set_mode((width, height))

# Colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Fonts
medium_font = pygame.font.Font("OpenSans-Regular.ttf", 28)
large_font = pygame.font.Font("OpenSans-Regular.ttf", 40)
move_font = pygame.font.Font("OpenSans-Regular.ttf", 60)

# Game state constants
X = "X"
O = "O"
EMPTY = None

# Initialize the game board
def init_board():
    return [[EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY], [EMPTY, EMPTY, EMPTY]]

# Check if there is a winner
def get_winner(board):
    for row in board:
        if row[0] == row[1] == row[2] != EMPTY:
            return row[0]

    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] != EMPTY:
            return board[0][col]

    if board[0][0] == board[1][1] == board[2][2] != EMPTY:
        return board[0][0]

    if board[0][2] == board[1][1] == board[2][0] != EMPTY:
        return board[0][2]

    return None

# Get available moves (empty spaces on the board)
def available_moves(board):
    return [(r, c) for r in range(3) for c in range(3) if board[r][c] == EMPTY]

# Apply a move to the board
def apply_move(board, move, player):
    new_board = [row[:] for row in board]
    new_board[move[0]][move[1]] = player
    return new_board

# AI's move decision using the minimax algorithm
def minimax(board, is_maximizing):
    winner = get_winner(board)
    if winner == X:
        return 1
    elif winner == O:
        return -1
    elif not any(EMPTY in row for row in board):
        return 0

    if is_maximizing:
        max_eval = float('-inf')
        for move in available_moves(board):
            eval = minimax(apply_move(board, move, X), False)
            max_eval = max(max_eval, eval)
        return max_eval
    else:
        min_eval = float('inf')
        for move in available_moves(board):
            eval = minimax(apply_move(board, move, O), True)
            min_eval = min(min_eval, eval)
        return min_eval

def get_best_move(board):
    best_move = None
    best_value = float('-inf')
    for move in available_moves(board):
        move_value = minimax(apply_move(board, move, X), False)
        if move_value > best_value:
            best_value = move_value
            best_move = move
    return best_move

# Main game loop
def main():
    user = None
    board = init_board()
    ai_turn = False
    while True:
        screen.fill(BLACK)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        if user is None:
            # Choose Player
            title = large_font.render("Play Tic-Tac-Toe", True, WHITE)
            title_rect = title.get_rect()
            title_rect.center = (width / 2, 50)
            screen.blit(title, title_rect)

            # Create buttons for choosing X or O
            play_x_button = pygame.Rect(width / 8, height / 2, width / 4, 50)
            play_x_text = medium_font.render("Play as X", True, BLACK)
            play_x_rect = play_x_text.get_rect()
            play_x_rect.center = play_x_button.center
            pygame.draw.rect(screen, WHITE, play_x_button)
            screen.blit(play_x_text, play_x_rect)

            play_o_button = pygame.Rect(5 * width / 8, height / 2, width / 4, 50)
            play_o_text = medium_font.render("Play as O", True, BLACK)
            play_o_rect = play_o_text.get_rect()
            play_o_rect.center = play_o_button.center
            pygame.draw.rect(screen, WHITE, play_o_button)
            screen.blit(play_o_text, play_o_rect)

            # Check for button click to select X or O
            click, _, _ = pygame.mouse.get_pressed()
            if click == 1:
                mouse_pos = pygame.mouse.get_pos()
                if play_x_button.collidepoint(mouse_pos):
                    time.sleep(0.2)
                    user = X
                elif play_o_button.collidepoint(mouse_pos):
                    time.sleep(0.2)
                    user = O
        else:
            # Draw board
            tile_size = 80
            tile_origin = (width / 2 - (1.5 * tile_size), height / 2 - (1.5 * tile_size))
            tiles = []
            for i in range(3):
                row = []
                for j in range(3):
                    rect = pygame.Rect(tile_origin[0] + j * tile_size, tile_origin[1] + i * tile_size, tile_size, tile_size)
                    pygame.draw.rect(screen, WHITE, rect, 3)

                    if board[i][j] != EMPTY:
                        move = move_font.render(board[i][j], True, WHITE)
                        move_rect = move.get_rect()
                        move_rect.center = rect.center
                        screen.blit(move, move_rect)
                    row.append(rect)
                tiles.append(row)

            game_over = get_winner(board) is not None or not any(EMPTY in row for row in board)
            current_player = X if board.count(X) <= board.count(O) else O

            # Display game status
            if game_over:
                winner = get_winner(board)
                status_text = f"Game Over: Tie." if winner is None else f"Game Over: {winner} wins."
            elif user == current_player:
                status_text = f"Play as {user}"
            else:
                status_text = "Computer thinking..."

            status = large_font.render(status_text, True, WHITE)
            status_rect = status.get_rect()
            status_rect.center = (width / 2, 30)
            screen.blit(status, status_rect)

            # Handle AI move
            if user != current_player and not game_over:
                if ai_turn:
                    time.sleep(0.5)
                    move = get_best_move(board)
                    board = apply_move(board, move, X)
                    ai_turn = False
                else:
                    ai_turn = True

            # Handle user move
            click, _, _ = pygame.mouse.get_pressed()
            if click == 1 and user == current_player and not game_over:
                mouse_pos = pygame.mouse.get_pos()
                for i in range(3):
                    for j in range(3):
                        if board[i][j] == EMPTY and tiles[i][j].collidepoint(mouse_pos):
                            board = apply_move(board, (i, j), user)

            # Play Again button
            if game_over:
                again_button = pygame.Rect(width / 3, height - 65, width / 3, 50)
                again_text = medium_font.render("Play Again", True, BLACK)
                again_rect = again_text.get_rect()
                again_rect.center = again_button.center
                pygame.draw.rect(screen, WHITE, again_button)
                screen.blit(again_text, again_rect)
                click, _, _ = pygame.mouse.get_pressed()
                if click == 1:
                    mouse_pos = pygame.mouse.get_pos()
                    if again_button.collidepoint(mouse_pos):
                        time.sleep(0.2)
                        user = None
                        board = init_board()

        pygame.display.flip()

if __name__ == "__main__":
    main()
